In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import io

sns.set_theme(style="whitegrid")

df = pd.read_csv(io.StringIO('results/my_specific_file.csv'))

# Convert Unix timestamp (ms) to Datetime object
df['Datetime'] = pd.to_datetime(df['Time'], unit='ms')

In [ ]:
temp_cols = [col for col in df.columns if 'CPU_TEMP_' in col]
df_temp = df.melt(id_vars=['Datetime'], value_vars=temp_cols, var_name='Core', value_name='Temperature')
df_temp['Core'] = df_temp['Core'].str.replace('CPU_TEMP_', '')

usage_cols = [col for col in df.columns if 'CPU_USAGE_' in col]
df_usage = df.melt(id_vars=['Datetime'], value_vars=usage_cols, var_name='Core', value_name='Usage')
df_usage['Core'] = df_usage['Core'].str.replace('CPU_USAGE_', '')

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Temperature Plots
sns.boxplot(x='Core', y='Temperature', data=df_temp, ax=axes[0, 0], palette="Reds")
axes[0, 0].set_title('CPU Temperature by Core (Box Plot)')

sns.violinplot(x='Core', y='Temperature', data=df_temp, ax=axes[0, 1], palette="Reds", inner="quartile")
axes[0, 1].set_title('CPU Temperature by Core (Violin Plot)')

# Usage Plots
sns.boxplot(x='Core', y='Usage', data=df_usage, ax=axes[1, 0], palette="Blues")
axes[1, 0].set_title('CPU Usage by Core (Box Plot)')

sns.violinplot(x='Core', y='Usage', data=df_usage, ax=axes[1, 1], palette="Blues", inner="quartile")
axes[1, 1].set_title('CPU Usage by Core (Violin Plot)')

plt.tight_layout()
plt.show()

In [ ]:
print("-" * 50)
print("STATISTICAL ANALYSIS")
print("-" * 50)

# Kruskal-Wallis Test
temp_groups = [df[col].dropna() for col in temp_cols]
h_stat_temp, p_val_temp = stats.kruskal(*temp_groups)
print(f"Kruskal-Wallis Test (Temperature): H-Statistic = {h_stat_temp:.2f}, p-value = {p_val_temp:.4e}")

if p_val_temp < 0.05:
    print("-> Conclusion: There is a statistically significant difference in temperature between the CPU cores.\n")
else:
    print("-> Conclusion: No significant difference in temperature across the cores.\n")

# Pearson Correlation Matrix
df['Avg_CPU_Usage'] = df[usage_cols].mean(axis=1)
corr_data = df[['Avg_CPU_Usage', 'USED_MEMORY', 'SYSTEM_POWER (Watts)']]
correlation_matrix = corr_data.corr(method='pearson')

print("Pearson Correlation Matrix (System Load vs Power):")
print(correlation_matrix)

# Heatmap of correlation
plt.figure(figsize=(6, 4))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, fmt=".2f")
plt.title('Correlation Matrix Heatmap')
plt.show()